In [745]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional 

INITIAL_BALANCE = 5000

In [746]:
df = pd.read_excel(r"C:\Users\maxiv\Downloads\TTP_17-12-2025_12.47.xlsx",sheet_name="All")
df = pd.read_excel(r"C:\Users\maxiv\Downloads\the5ers_bots.xlsx",sheet_name="All")

#Rellenando na con ceros 
df.fillna(0,inplace=True)


#Colocando comment en operaciones manuales
df.loc[df["Comment"]==0,"Comment"] = "Manual"

#Resetando indice
df.reset_index(inplace=True,drop=True)

#Rename a dos columnas que se llamaban iguales
df.rename(columns={"Price":"Price Entry", "Price.1":"Price Exit"},inplace=True)
df.rename(columns={"Time":"Time Entry","Time.1":"Time Exit"},inplace=True)

#Nueva columna con el balance
df["Balance"] = INITIAL_BALANCE + (df["Profit"]+df["Commission"]+df["Swap"]).cumsum() 

#Nueva columna con el profit despues de swaps y comisiones
df["Profit-Commi-Swap"] = df["Profit"] + df["Commission"]+ df["Swap"]


In [747]:
def get_bots_by_comment():
    return df["Comment"].unique()

def handle_errors_not_string_and_not_in_df(x):
     #Verificicar que sea none para no entrar en el if y que sea una string 
    if x is not None:
        if  not isinstance(x,str) :
            raise TypeError(f"{x} no es una string")
    
        #Conseguir los bots por comentario del df y verificar si bot se encuentra
        if x not in get_bots_by_comment():
            raise KeyError(f"{x} no está en el dataframe")
        

In [748]:
def win_rate(df:pd.DataFrame,bot:Optional[str] = None, grafic:Optional[bool] = False):
    """
    Calcula los ratios para un bot en especifico o para el total de la cuenta
    Colocar el comentario del bot
    
    Nota: Se considera BE cuando el profit fue menor que el 0.15% del balance actual
    """
    
    handle_errors_not_string_and_not_in_df(bot)

    #Para todos los datos
    if bot is None:         
        pass

    #Para un bot en especifico
    else:         
        df= df[df["Comment"]==bot]


    breakeven =  sum((df["Profit"] < df["Balance"]*0.0015) & (df["Profit"] > 0))
    
    win = sum((df["Profit"] > 0) & (df["Profit"] > df["Balance"]*0.0015)) 
    
    losses = sum(df["Profit"] < 0 )
    
    total = df["Profit"].count()
    
    print ( f'{'Ratio total:' if (bot is None) else f"Para el bot {bot} con {total} trades sus ratios son:"}'
            f'\nWin:{round(win/total*100,2)}%\n' 
            f'Loss:{round(losses/total*100,2)}%\n' 
            f'Breakeven: {round(breakeven/total*100,2)}%\n')


    ##Grafico
    # Crear el gráfico
    if grafic == True:

        plt.pie(
            [win,losses,breakeven],
            labels=["Win","Loss","Be"],
            autopct='%1.2f%%'
            )
        plt.title(f'{bot if bot is not None else "Ratios"}')
        plt.show()



In [749]:
def profit_factor(df:pd.DataFrame,bot:Optional[str]=None):
    """
    Calcula el profit factor para un bot en especifico o para el total de la cuenta
    Colocar el comentario del bot    
    
    Nota: No se consideran BE y se toma el beneficio sin swaps y comisiones
    """
   
    handle_errors_not_string_and_not_in_df(bot)
    #Para todos los datos

    if bot is None:         
        pass 

    #Para un bot en especifico
    else:         
        df= df[df["Comment"]==bot]


    total_profit = df.query("Profit >= 0")["Profit"].sum()
    total_loss = df.query("Profit < 0")["Profit"].sum()
    
    if total_loss!=0:
        return print(
            f"Profit Factor{(" de " + bot) if bot is not None else ""}: {round(-1*total_profit/total_loss,2)} "
          )

    else:
        print("Total de perdidas = 0, no se puede realizar la división")


In [750]:
def profit_factor_v2(data,by_bot=True):
    
    #Profit factor por bot
    if by_bot:
        x = df.groupby("Comment")        

        return x.apply(profit_factor_v2,by_bot=False,include_groups=False) #type: ignore


    total_profit = data[data["Profit"]>=0]["Profit"].sum()
    total_loss = data[data["Profit"]< 0]["Profit"].sum()

    #Division por 0
    if total_loss==0:
        return float('inf')
    

    return round(abs(total_profit/total_loss),2)
    



In [751]:
def win_rate_v2(data:pd.DataFrame,by_bot = False,BE = True):
    """
    Calcula los ratios para los bots o para el total de la cuenta
    Se puede desactivar el calculo con BE y hacerlo normal

    Nota: Se considera BE cuando el profit fue menor que el 0.15% del balance actual
    """

    #data es un dataframe?
    if not isinstance(data,pd.DataFrame):
        raise TypeError("data is not a Dataframe")
    
    #Profit y balance son columnas necesarias para el calculo, están?
    if "Profit" not in data.columns or "Balance" not in data.columns:
        raise KeyError("Columns 'Profit' or 'Balance' is not in your dataframe")

  

    #Calculo
    total = data["Profit"].count()

    percent = lambda x: round(x*100/total,2)  

    win = ((data["Profit"] > 0) & (data["Profit"] > data["Balance"]*0.0015)).sum() 
    win_percent = percent(win)

    losses = (data["Profit"] < 0 ).sum()
    losses_percent = percent(losses)
    
    #Diccionario base
    dicc =  {"Win":[f"{win_percent}%",win],
            "Loss":[f"{losses_percent}%",losses]}

    #Queres los breakeven?
    if BE:
        breakeven =  ((data["Profit"] < data["Balance"]*0.0015) & (data["Profit"] > 0)).sum()
        breakeven_percent = percent(breakeven)

        dicc["BE"] = [f"{breakeven_percent}%",breakeven]
    
    #Añado el total
    
    dicc["Total"] = ["100%", total]


    #Para cada bot
    if by_bot:
        x = data.groupby("Comment")[["Profit","Balance"]]
        return x.apply(win_rate_v2,BE=BE)
    

    return pd.DataFrame(dicc,index=["Percent","n"])

In [752]:
profit = profit_factor_v2(df).to_frame("Profit Factor") #type: ignore


df_win = win_rate_v2(df,by_bot=True)

df_win_flat = df_win.unstack(level=-1)
df_win_flat.columns = [f"{col[0]}_{col[1]}" for col in df_win_flat.columns]




df_final = pd.merge(profit, df_win_flat, left_index=True, right_index=True, how="outer")

df_final["Profit"] = df.groupby("Comment")["Profit-Commi-Swap"].sum()
df_final.sort_values(by="Profit",inplace=True)
df_final

,Profit Factor,Win_Percent,Win_n,Loss_Percent,Loss_n,BE_Percent,BE_n,Total_Percent,Total_n,Profit
Comment,,,,,,,,,,
Manual,0.00,0.0%,0,100.0%,2,0.0%,0,100%,2,-89.69
SP_Strategy_5_8_16,0.00,0.0%,0,100.0%,1,0.0%,0,100%,1,-34.74
USDJPY_Strategy_2_12_24,0.00,0.0%,0,100.0%,2,0.0%,0,100%,2,-19.22
NQ_WF_Strategy_4_10_26_1_1,inf,100.0%,1,0.0%,0,0.0%,0,100%,1,8.12
Porfolio 1.2.3 with SL,4.42,66.67%,2,33.33%,1,0.0%,0,100%,3,91.72
XAUUSD_M5,1.38,33.33%,3,44.44%,4,22.22%,2,100%,9,98.84
